In [1]:
from importlib import reload
from tqdm import tqdm
import HateSpeechAggregator as hsa
from glob import glob
from datetime import datetime

reload(hsa);

files = glob('../clean-data/comments/*.csv')
files.sort()

# today = datetime.now()
# today_str = today.strftime("%Y%m%d")
today_str="20200709"
print("Today Date : ",today_str)  
for f in tqdm(files):
    split_file = str(f).split('/')[3]
    date_check_file = split_file.split("_")[-1]
    date_file=date_check_file.split(".")[0]
    if date_file==today_str:
        file_check =glob(f)
        hsa.run(file_check)

/home/curiousgirl/Public/Desktop/hate-speech-new/hsle/modules/CrowdTangleExportCommentsTools/CrowdTangleExportCommentsTools/HsleCandidateGenerationUtils.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp['label_alternative_spelling'] = [
0it [00:00, ?it/s]

Today Date :  20200709


## Calculate X2

In [2]:
from scipy.stats import chi2_contingency
import pandas as pd
from glob import glob

aggredate_id = str(max([int(f.split('/')[-1]) for f in glob('../../dashboard/clean-data/aggregated/*')]))
print(aggredate_id)
print('###############', aggredate_id)
print(
    chi2_contingency(pd.read_csv(
        '../clean-data/aggregated/'+aggredate_id+'/hspost-effect.csv', sep='~').groupby(
            ['post_has_lex','comment_has_lex'])['count'].sum().unstack())[:3],
    '\n',
    chi2_contingency(pd.read_csv(
        '../clean-data/aggregated/'+aggredate_id+'/hsfirst-comment-effect.csv', sep='~').groupby(
            ['first_comment_has_lex','reply_has_lex'])['count'].sum().unstack())[:3]
)

20200711230223
############### 20200711230223


FileNotFoundError: [Errno 2] File ../clean-data/aggregated/20200711230223/hspost-effect.csv does not exist: '../clean-data/aggregated/20200711230223/hspost-effect.csv'

## Calculate Coorelation between #HS-#UniqueAccounts

In [ ]:
df1 = pd.read_csv('../clean-data/aggregated/'+aggredate_id+'/lex-topic-page-time.csv', sep='~')
df1['key'] = ['{}-{}'.format(a,b) for a,b in zip(df1.Page, df1.Date)]
display(df1.head(2))

df2 = pd.read_csv('../clean-data/aggregated/'+aggredate_id+'/page-reach.csv', sep='~')
df2['key'] = ['{}-{}'.format(a,b) for a,b in zip(df2.page_group_name, df2.date)]
display(df2.head(2))

In [ ]:
df = df1.merge(df2, how='left', on='key').loc[:, ['Page','Topic','Date','count','Profile ID']]
print(df.shape)
df.head(2)

In [ ]:
x = df1.groupby('Page')['Page'].count()
x.name = 'hs'
y = df2.groupby('page_group_name')['count'].sum()
y.name = 'ua'

In [ ]:
z = pd.DataFrame([x,y]).T.fillna(0)

In [ ]:
import seaborn as sns

ax = sns.scatterplot(x='ua', y='hs', data=z)
ax.set(xscale="log", yscale="log")